In [1]:
import numpy as np
from collections import deque

# Part 1

In [2]:
test = np.array(['--- scanner 0 ---',
                 '404,-588,-901',
                 '528,-643,409',
                 '-838,591,734',
                 '390,-675,-793',
                 '-537,-823,-458',
                 '-485,-357,347',
                 '-345,-311,381',
                 '-661,-816,-575',
                 '-876,649,763',
                 '-618,-824,-621',
                 '553,345,-567',
                 '474,580,667',
                 '-447,-329,318',
                 '-584,868,-557',
                 '544,-627,-890',
                 '564,392,-477',
                 '455,729,728',
                 '-892,524,684',
                 '-689,845,-530',
                 '423,-701,434',
                 '7,-33,-71',
                 '630,319,-379',
                 '443,580,662',
                 '-789,900,-551',
                 '459,-707,401',

                 '--- scanner 1 ---',
                 '686,422,578',
                 '605,423,415',
                 '515,917,-361',
                 '-336,658,858',
                 '95,138,22',
                 '-476,619,847',
                 '-340,-569,-846',
                 '567,-361,727',
                 '-460,603,-452',
                 '669,-402,600',
                 '729,430,532',
                 '-500,-761,534',
                 '-322,571,750',
                 '-466,-666,-811',
                 '-429,-592,574',
                 '-355,545,-477',
                 '703,-491,-529',
                 '-328,-685,520',
                 '413,935,-424',
                 '-391,539,-444',
                 '586,-435,557',
                 '-364,-763,-893',
                 '807,-499,-711',
                 '755,-354,-619',
                 '553,889,-390',

                 '--- scanner 2 ---',
                 '649,640,665',
                 '682,-795,504',
                 '-784,533,-524',
                 '-644,584,-595',
                 '-588,-843,648',
                 '-30,6,44',
                 '-674,560,763',
                 '500,723,-460',
                 '609,671,-379',
                 '-555,-800,653',
                 '-675,-892,-343',
                 '697,-426,-610',
                 '578,704,681',
                 '493,664,-388',
                 '-671,-858,530',
                 '-667,343,800',
                 '571,-461,-707',
                 '-138,-166,112',
                 '-889,563,-600',
                 '646,-828,498',
                 '640,759,510',
                 '-630,509,768',
                 '-681,-892,-333',
                 '673,-379,-804',
                 '-742,-814,-386',
                 '577,-820,562',

                 '--- scanner 3 ---',
                 '-589,542,597',
                 '605,-692,669',
                 '-500,565,-823',
                 '-660,373,557',
                 '-458,-679,-417',
                 '-488,449,543',
                 '-626,468,-788',
                 '338,-750,-386',
                 '528,-832,-391',
                 '562,-778,733',
                 '-938,-730,414',
                 '543,643,-506',
                 '-524,371,-870',
                 '407,773,750',
                 '-104,29,83',
                 '378,-903,-323',
                 '-778,-728,485',
                 '426,699,580',
                 '-438,-605,-362',
                 '-469,-447,-387',
                 '509,732,623',
                 '647,635,-688',
                 '-868,-804,481',
                 '614,-800,639',
                 '595,780,-596',

                 '--- scanner 4 ---',
                 '727,592,562',
                 '-293,-554,779',
                 '441,611,-461',
                 '-714,465,-776',
                 '-743,427,-804',
                 '-660,-479,-426',
                 '832,-632,460',
                 '927,-485,-438',
                 '408,393,-506',
                 '466,436,-512',
                 '110,16,151',
                 '-258,-428,682',
                 '-393,719,612',
                 '-211,-452,876',
                 '808,-476,-593',
                 '-575,615,604',
                 '-485,667,467',
                 '-680,325,-822',
                 '-627,-443,-432',
                 '872,-547,-609',
                 '833,512,582',
                 '807,604,487',
                 '839,-516,451',
                 '891,-625,532',
                 '-652,-548,-490',
                 '30,-46,-14'])

In [3]:
def process_input(data):
    scanners = []
    beacons = []
    for line in data[1:]:
        if line[:3] == '---':
            scanners.append(beacons)
            beacons = []
        else:
            x, y, z = line.split(',')
            beacons.append(np.array([int(x), int(y), int(z)]))
    scanners.append(np.array(beacons))
    return scanners

def beacon_displacements(scanner):
    displace = []
    for i in range(0, len(scanner)):
        beacons = []
        for j in range(0, len(scanner)):
            dxdydz = scanner[i] - scanner[j]
            beacons.append(dxdydz)
        displace.append(np.array(beacons))
    return displace

def get_transformations():
    t_x = np.array([np.array([ 1, 0, 0]),
                    np.array([ 0, 0,-1]),
                    np.array([ 0, 1, 0])])
    t_y = np.array([np.array([ 0, 0, 1]),
                    np.array([ 0, 1, 0]),
                    np.array([-1 ,0, 0])])
    t_z = np.array([np.array([ 0,-1, 0]),
                    np.array([ 1, 0, 0]),
                    np.array([ 0, 0, 1])])
    
    return t_x, t_y, t_z

def transform_beacons(beacons, trans):
    for i in range(0, len(beacons)):
        beacons[i] = np.dot(trans, beacons[i])
    return beacons

def count_matches(set00, set11):
    for set0 in set00:
        for set1 in set11:
            count = 0
            for s0 in set0:
                for s1 in set1:
                    if s0[0] == s1[0] and s0[1] == s1[1] and s0[2] == s1[2]:
                        count += 1
            if count >= 12:
                return count
    return 0

def add_to_known_beacons(known_beacons, reference, differ, scanners, scan, locations, ref):
    origin = scanners[ref]
    new = scanners[scan]
    
    for i in range(0, len(reference)):
        set0 = reference[i]
        for j in range(0, len(differ)):
            set1 = differ[j]
            count = 0
            for s0 in set0:
                for s1 in set1:
                    if s0[0] == s1[0] and s0[1] == s1[1] and s0[2] == s1[2]:
                        count += 1
            if count >= 12:
                #we have the transformation
                x0y0z0 = origin[i]
                x1y1z1 = new[j]
                
                dx = origin[i][0]-new[j][0]
                dy = origin[i][1]-new[j][1]
                dz = origin[i][2]-new[j][2]
                
                locations[scan] = np.array([dx, dy, dz])
                
                for k in range(0, len(new)):
                    new[k][0] += dx
                    new[k][1] += dy
                    new[k][2] += dz
                    isin = False
                    for l in range(0, len(known_beacons)):
                        if new[k][0] == known_beacons[l][0] and new[k][1] == known_beacons[l][1] and\
                                new[k][2] == known_beacons[l][2]:
                            isin = True
                            break
                    if isin == False:
                        known_beacons.append(new[k])
                        
                return known_beacons, locations
    return known_beacons, locations

def run_match(scanners, scan, reference, ref, locations, known_beacons):
    differ = beacon_displacements(scanners[scan])
    count = count_matches(reference, differ)
    if count >= 12:
        #add to known_beacons
        known_beacons, locations = add_to_known_beacons(known_beacons, reference, differ, scanners, scan, locations, ref)
        return True, scanners, reference, locations, known_beacons
    return False, scanners, reference, locations, known_beacons
    
def align_beacons(data, ref=0):
    scanners = process_input(data)
    trans = get_transformations()
    
    ref_queue = deque([ref])    
    locations = {0:np.array([0,0,0])}
    known_beacons = []
    for beacon in scanners[ref]:
        known_beacons.append(beacon)
    
    while len(ref_queue):
        ref = ref_queue.popleft()
        reference = beacon_displacements(scanners[ref])
        
        queue = deque(range(0, len(scanners)))
        for key in locations:
            queue.remove(key)

        while len(queue):
            scan = queue.popleft()
            fit = False


            for i in range(0, 4):
                scanners[scan] = transform_beacons(scanners[scan], trans[0])
                fit, scanners, reference, locations, known_beacons = run_match(scanners, scan, reference, ref, locations, known_beacons)
                if fit:
                    ref_queue.append(scan)
                    break

            if fit == False:
                for j in range(0, 3):
                    scanners[scan] = transform_beacons(scanners[scan], trans[1])
                    for i in range(0, 4):
                        scanners[scan] = transform_beacons(scanners[scan], trans[0])
                        fit, scanners, reference, locations, known_beacons = run_match(scanners, scan, reference, ref, locations, known_beacons)
                        if fit:
                            ref_queue.append(scan)
                            break
                    if fit:
                        break

            if fit == False:
                for j in range(0, 3):
                    scanners[scan] = transform_beacons(scanners[scan], trans[2])
                    if j == 1:
                        continue
                    for i in range(0, 4):
                        scanners[scan] = transform_beacons(scanners[scan], trans[0])
                        fit, scanners, reference, locations, known_beacons = run_match(scanners, scan, reference, ref, locations, known_beacons)
                        if fit:
                            ref_queue.append(scan)
                            break
                    if fit:
                        break
            
    return known_beacons, locations

print(len(align_beacons(test)[0]))

79


In [4]:
inpt = np.genfromtxt('day19_input.txt', dtype=str, delimiter='\n')
beacons, scanners = align_beacons(inpt)
print('Part 1  Result:', len(beacons))

Part 1  Result: 400


# Part 2

In [5]:
def manhattan(scanners):
    size = len(scanners.keys())
    largest = 0
    for i in range(0, size-1):
        for j in range(i+1, size):
            dist = abs(scanners[i][0]-scanners[j][0])+abs(scanners[i][1]-scanners[j][1])+abs(scanners[i][2]-scanners[j][2])
            if dist > largest:
                largest = dist
    return largest

print(manhattan(align_beacons(test)[1]))

3621


In [6]:
print('Part 2 Result:', manhattan(scanners))

Part 2 Result: 12168
